In [1]:
import calendar
import itertools

from operator import add
from operator import floordiv

import numpy as np

import toolz
from toolz import flip
from toolz import curry
from toolz import compose


import matplotlib
matplotlib.use('WXAgg')

import matplotlib.pyplot as plt
plt.ion()

import dataset.day

In [2]:
add      = curry(add)
floordiv = curry(floordiv)

floordivby = compose(curry, flip)(floordiv)

eager_map = compose(curry, compose)(list, map)

eager_zip   = compose(list, zip)
eager_chain = compose(list, itertools.chain)

# Generate for an arbitrary number of days

In [3]:
days = [dataset.day.generate_day_dataset(polling_interval = dataset.day.SUBDIVISION_MINS,
                                         min_traffic = dataset.day.MIN_TRAFFIC_PER_MINUTE,
                                         max_traffic = dataset.day.MAX_TRAFFIC_PER_MINUTE,
                                         max_traffic_at_peak = dataset.day.MAX_TRAFFIC_PER_MINUTE_AT_PEAK,
                                         peak_duration = dataset.day.PEAK_DURATION_MINS,
                                         peak_times = dataset.day.PEAKS)
        for i
        in range(31)]

In [4]:
plt.figure(1)

for day_ind, day in enumerate(days):
    plt.subplot(5, 7, day_ind + 1)
    plt.plot(day, color = 'b', linewidth = 1)
    plt.title(f'Day {day_ind + 1}')
    plt.xlabel('Time of day')
    plt.ylabel(f'Requests per {dataset.day.SUBDIVISION_MINS} minutes')
    plt.ylim([0, dataset.day.MAX_TRAFFIC_PER_MINUTE_AT_PEAK * dataset.day.SUBDIVISION_MINS * 1.25])
    plt.xticks(*dataset.day.time_steps, rotation = 60)
    plt.grid(True, linestyle = '--')

plt.show()

# Generate for one week

In [12]:
week = eager_chain(*[dataset.day.generate_day_dataset(polling_interval = dataset.day.SUBDIVISION_MINS,
                                                      min_traffic = dataset.day.MIN_TRAFFIC_PER_MINUTE,
                                                      max_traffic = dataset.day.MAX_TRAFFIC_PER_MINUTE,
                                                      max_traffic_at_peak = dataset.day.MAX_TRAFFIC_PER_MINUTE_AT_PEAK,
                                                      peak_duration = dataset.day.PEAK_DURATION_MINS,
                                                      peak_times = dataset.day.PEAKS)
                     for i
                     in range(7)])

time_steps = eager_zip(*[(t, timestamp) 
                        for t, timestamp 
                        in zip(*dataset.day.time_steps) 
                        if int(timestamp.split(':')[0]) % 4 == 0])

time_steps = eager_chain(*[eager_map(add(dataset.day.cycle_mins * day // dataset.day.SUBDIVISION_MINS), time_steps.copy()[0]) 
                           for day 
                           in range(7)])

time_stamps = eager_chain(*[[f'Day {i+1} - 00:00'] + dataset.day.time_steps[1][::4][1:] 
                             for i 
                             in range(7)])


print(time_steps)
print(time_stamps)

plt.plot(week)
plt.title(f'Week')
plt.xlabel('Time')
plt.ylabel(f'Requests per {dataset.day.SUBDIVISION_MINS} minutes')
plt.ylim([0, dataset.day.MAX_TRAFFIC_PER_MINUTE_AT_PEAK * dataset.day.SUBDIVISION_MINS * 1.25])
plt.xticks(time_steps, time_stamps, rotation = 90)
plt.grid(True, linestyle = '--')


[0, 48, 96, 144, 192, 240, 288, 336, 384, 432, 480, 528, 576, 624, 672, 720, 768, 816, 864, 912, 960, 1008, 1056, 1104, 1152, 1200, 1248, 1296, 1344, 1392, 1440, 1488, 1536, 1584, 1632, 1680, 1728, 1776, 1824, 1872, 1920, 1968]
['Day 1 - 00:00', '04:00', '08:00', '12:00', '16:00', '20:00', 'Day 2 - 00:00', '04:00', '08:00', '12:00', '16:00', '20:00', 'Day 3 - 00:00', '04:00', '08:00', '12:00', '16:00', '20:00', 'Day 4 - 00:00', '04:00', '08:00', '12:00', '16:00', '20:00', 'Day 5 - 00:00', '04:00', '08:00', '12:00', '16:00', '20:00', 'Day 6 - 00:00', '04:00', '08:00', '12:00', '16:00', '20:00', 'Day 7 - 00:00', '04:00', '08:00', '12:00', '16:00', '20:00']


# Generate for one month

In [5]:
year  = 2018
month = 2
weekday_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

month_start_weekday, month_length = calendar.monthrange(year, month)

month_day_traffic = [dataset.day.generate_day_dataset(polling_interval = dataset.day.SUBDIVISION_MINS,
                                                      min_traffic = dataset.day.MIN_TRAFFIC_PER_MINUTE,
                                                      max_traffic = dataset.day.MAX_TRAFFIC_PER_MINUTE,
                                                      max_traffic_at_peak = dataset.day.MAX_TRAFFIC_PER_MINUTE_AT_PEAK,
                                                      peak_duration = dataset.day.PEAK_DURATION_MINS,
                                                      peak_times = dataset.day.PEAKS)
                     for i
                     in range(month_length)]

time_steps  = dataset.day.time_steps[0][::12]
time_stamps = dataset.day.time_steps[1][::12]

plt.figure(1)
plt.subplots_adjust(wspace = 0.35, hspace = 0.35)

plt.suptitle(f'{month_names[month - 1]}, {year}', fontsize = 20, fontweight='bold')

for i, weekday_name in enumerate(weekday_names):
    plt.subplot(5, 7, i + 1)
    plt.title(weekday_name, fontsize = 15, fontweight='bold')
    if i < month_start_weekday:
        plt.yticks([], [])
        plt.xticks([], [])

for day_ind, day in enumerate(month_day_traffic):
    plt.subplot(5, 7, month_start_weekday + day_ind + 1)
    plt.plot(day, color = 'b', linewidth = 0.5)
    plt.ylim([0, dataset.day.MAX_TRAFFIC_PER_MINUTE_AT_PEAK * dataset.day.SUBDIVISION_MINS * 1.25])
    plt.xticks(time_steps, time_stamps, rotation = 0)
    plt.grid(True, linestyle = '--')
    
for i in range(len(month_day_traffic) + month_start_weekday, 5*7):
    plt.subplot(5, 7, i + 1)
    plt.yticks([], [])
    plt.xticks([], [])

plt.show()

e:\tools\languages\python36\lib\site-packages\matplotlib\cbook\deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)
